In [15]:
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import matplotlib.lines as mlines
import matplotlib.markers as mmarkers

matplotlib.rcParams['figure.dpi'] = 400

In [16]:
df = pd.read_csv("all_failures_database.csv")

In [36]:
grouped = df.groupby(["board", "family", "config", "bfm", "hardening"]).sum()
grouped.loc[pd.IndexSlice[:, :, :, "FLIP_SINGLE_BIT", :], "#faults"] = 750
grouped.loc[pd.IndexSlice[:, :, :, "RANDOM_VALUE", :], "#faults"] = 750
grouped.loc[pd.IndexSlice[:, :, :, "WARP_RANDOM_VALUE", :], "#faults"] = 250
grouped["PVF Critical SDC"] = grouped["critical_SDC"] / grouped["#faults"]
grouped = grouped["PVF Critical SDC"].reset_index().groupby(["family", "bfm", "hardening"]).mean()

In [38]:
bfms = grouped.unstack("hardening")

In [39]:
bfms

PVF Critical SDC           
hardening                              Hardened Unhardened
family       bfm                                          
EVA          FLIP_SINGLE_BIT           0.002333   0.016333
             RANDOM_VALUE              0.024667   0.210667
             WARP_RANDOM_VALUE         0.035000   0.324000
MaxViT       FLIP_SINGLE_BIT           0.006333   0.017667
             RANDOM_VALUE              0.100333   0.213000
             WARP_RANDOM_VALUE         0.264000   0.432000
Original ViT FLIP_SINGLE_BIT           0.002133   0.009867
             RANDOM_VALUE              0.040533   0.098133
             WARP_RANDOM_VALUE         0.063600   0.153200
Swin         FLIP_SINGLE_BIT           0.005111   0.006667
             RANDOM_VALUE              0.022889   0.030222
             WARP_RANDOM_VALUE         0.050000   0.057333

In [40]:
# plot initialization

font_size = 14
col_order = ["Original", "EVA", "Swin", "MaxViT"]


In [41]:
chart = sns.scatterplot(data=bfms, x="Family Model", y="Avg. Critical SDC", style="BFM", hue="Type", palette="flare", s=150,)

# avoid collision between points
for points in chart.collections:
    vertices = points.get_offsets().data
    if len(vertices) > 0:
        vertices[:, 0] += np.random.uniform(-0.3, 0.3, vertices.shape[0])
        points.set_offsets(vertices)

# setting axis labels
chart.set(xlabel=None)
chart.set_ylabel(ylabel="Average Critical SDC", fontsize=font_size)
plt.yticks(fontsize=font_size)
plt.xticks(fontsize=font_size)

# legend management
handles, labels = chart.get_legend_handles_labels()
handles_type = handles[1:3]
handles_bfm = handles[4:]

for hd in handles_type:
    hd.set_label("Hardened" if "H." == hd.get_label() else "Unhardened")

bfms_lgd = plt.legend(handles=handles_bfm, loc="upper center", bbox_to_anchor=(.50,1.11), ncols=len(handles_bfm), frameon=False,
                      fontsize=font_size, columnspacing=0.01, handletextpad=0.01)
for lg_hd in bfms_lgd.legendHandles:
    lg_hd._sizes = [100]
    
type_lgd = plt.legend(handles=handles_type, loc="upper center", bbox_to_anchor=(.50,1.18), ncols=len(handles_type), frameon=False,
           fontsize=font_size, columnspacing=0.01, handletextpad=0.01)
for lg_hd in type_lgd.legendHandles:
    lg_hd._sizes = [100]

plt.gca().add_artist(bfms_lgd)

plt.gca().yaxis.set_major_formatter(mtick.FuncFormatter(lambda y, _: f'{int(y)}%')) 
chart.set_xlim(-0.5, 3.5)

# plotting lines between the categories
plt.axvline(x=.5, color='black', ls=":", lw=1.75)
plt.axvline(x=1.5, color='black', ls=":", lw=1.75)
plt.axvline(x=2.5, color='black', ls=":", lw=1.75)
plt.tight_layout()
# saving the plot

ValueError: Could not interpret value `Family Model` for parameter `x`

In [42]:
chart = sns.catplot(data=bfms, x="BFM", y="Avg. Critical SDC", hue="Type", col="Family Model", aspect=.4, col_order=col_order, legend=False)

# setting axis labels
chart.set(xlabel=None)
chart.set_titles(col_template='{col_name}')
chart.set_ylabels("Average Critical SDC", fontsize=font_size)
plt.xlabel("")
plt.yticks(fontsize=font_size)
plt.gca().yaxis.set_major_formatter(mtick.FuncFormatter(lambda y, _: f'{int(y)}%')) 
plt.xticks([])


plt.tight_layout()
plt.savefig("./avg_crit_sdc_catplot.pdf")

ValueError: Could not interpret input 'BFM'

In [43]:
chart = sns.stripplot(data=bfms,  x="Family Model", y="Avg. Critical SDC",  hue="Type", dodge=True, jitter=True)

# setting axis labels
chart.set(xlabel=None)

chart.set_ylabel("Average Critical SDC", fontsize=font_size)
plt.xlabel("")
plt.yticks(fontsize=font_size)
plt.gca().yaxis.set_major_formatter(mtick.FuncFormatter(lambda y, _: f'{int(y)}%')) 

plt.tight_layout()
plt.savefig("./avg_crit_sdc_catplot.pdf")

ValueError: Could not interpret input 'Family Model'

In [48]:
bfms = bfms.reindex(["Original ViT", "EVA", "Swin", "MaxViT"], level="family").reindex(["FLIP_SINGLE_BIT", "RANDOM_VALUE", "WARP_RANDOM_VALUE"], level="bfm")

In [49]:
bfms.to_excel("~/git_research/date_2024_vits/data/bfmgraph_tmp.xlsx")

In [50]:
bfms

PVF Critical SDC           
hardening                              Hardened Unhardened
family       bfm                                          
Original ViT FLIP_SINGLE_BIT           0.002133   0.009867
             RANDOM_VALUE              0.040533   0.098133
             WARP_RANDOM_VALUE         0.063600   0.153200
EVA          FLIP_SINGLE_BIT           0.002333   0.016333
             RANDOM_VALUE              0.024667   0.210667
             WARP_RANDOM_VALUE         0.035000   0.324000
Swin         FLIP_SINGLE_BIT           0.005111   0.006667
             RANDOM_VALUE              0.022889   0.030222
             WARP_RANDOM_VALUE         0.050000   0.057333
MaxViT       FLIP_SINGLE_BIT           0.006333   0.017667
             RANDOM_VALUE              0.100333   0.213000
             WARP_RANDOM_VALUE         0.264000   0.432000